In [1]:
import pandas as pd
import os
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from time import sleep
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pickle
from IPython.display import IFrame, display
import numpy as np

CLIENT_ID = os.getenv("spotipy_id")
CLIENT_SECRET = os.getenv("spotipy_secret")

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=CLIENT_ID,
                                                           client_secret=CLIENT_SECRET))
df = pd.read_csv('/Users/Barbara/Desktop/Ironhack/Labs/Week_6/spotipy-api/tracks_clustered_df.csv')
df_tf = pd.read_csv('/Users/Barbara/Desktop/Ironhack/Labs/Week_6/spotipy-api/tracks_and_features.csv')

/Users/Barbara/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Get the user's favorite song and artist
fav_song = input("What's your favorite song? ")
fav_song_artist = input("Who's the artist? ")

# Search for the song on Spotify
results = sp.search(q="track:"+fav_song+" artist:"+fav_song_artist, limit=1)

# Check if a track was found
if results['tracks']['items']:
    # Extract the track details
    track = results['tracks']['items'][0]
    track_name = track['name']
    track_artist = track['artists'][0]['name']
    track_id = track['id']

    # Display the Spotify embed before asking for confirmation
    display(IFrame(src=f"https://open.spotify.com/embed/track/{track_id}?utm_source=generator",
                   width="320",
                   height="80",
                   frameborder="0",
                   allowtransparency="true",
                   allow="encrypted-media"))

    # # Ask for confirmation
    # response = input('Is it the song? (Y/N) ')

    # if response.upper() == 'Y':
    #     # If confirmed, proceed with the recommendation logic
    #     # Assuming `tracks_clustered_df` is your DataFrame with track clusters
    #     # and `new_song_scaled_features` is the scaled features of the new song
    #     # prediction = km_100.predict(new_song_scaled_features)
        
    #     # # Get a recommendation from the same cluster
    #     # recommendation2 = tracks_clustered_df[tracks_clustered_df['cluster_km100'] == prediction[0]].sample(1)
    #     # recommended_track_id = recommendation2['track_id'].iloc[0]

    #     # # Display the recommended song
    #     # display(IFrame(src=f"https://open.spotify.com/embed/track/{recommended_track_id}?utm_source=generator",
    #     #                width="320",
    #     #                height="80",
    #     #                frameborder="0",
    #     #                allowtransparency="true",
    #     #                allow="encrypted-media"))
    # else:
    #     print('Are you sure the song exists? Try again.')
else:
    print('No matching song found. Please check the details and try again.')

In [3]:
def check_song_in_dataset(track_id, df):
    return track_id in df['track_id'].values

# Example usage
track_id_to_check = track_id
is_in_dataset = check_song_in_dataset(track_id_to_check, df)
print(f"Is the song ID '{track_id_to_check}' in the dataset? {is_in_dataset}")

Is the song ID '63OQupATfueTdZMWTxW03A' in the dataset? True


In [4]:
### with training the model

list_of_audio_features_fav=[]
list_of_audio_features_fav.extend(sp.audio_features(track_id))
list_of_audio_features_fav = list_of_audio_features_fav [0]

danceability = []
energy = []
key = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []
duration_ms = []
time_signature = []
track_href = []


danceability.append(list_of_audio_features_fav['danceability'])
energy.append(list_of_audio_features_fav['energy'])
key.append(list_of_audio_features_fav.get('key', -1))
loudness.append(list_of_audio_features_fav['loudness'])
mode.append(list_of_audio_features_fav['mode'])
speechiness.append(list_of_audio_features_fav['speechiness'])
acousticness.append(list_of_audio_features_fav['acousticness'])
instrumentalness.append(list_of_audio_features_fav['instrumentalness'])
liveness.append(list_of_audio_features_fav['liveness'])
valence.append(list_of_audio_features_fav['valence'])
tempo.append(list_of_audio_features_fav['tempo'])
duration_ms.append(list_of_audio_features_fav['duration_ms'])
time_signature.append(list_of_audio_features_fav['time_signature'])
track_href.append(list_of_audio_features_fav['track_href'])

audio_features_dict = {
    'danceability':danceability,
    'energy':energy,
    'key':key,
    'loudness':loudness,
    'mode':mode,
    'speechiness':speechiness,
    'acousticness':acousticness,
    'instrumentalness':instrumentalness,
    'liveness':liveness,
    'valence':valence,
    'tempo':tempo,
    'duration_ms':duration_ms,
    'time_signature':time_signature,
    'track_href':track_href
    }

audio_features_df = pd.DataFrame(audio_features_dict)

artist_ids = []
artist_names = []
track_ids = []
track_href_1 = []
track_names = []
album_release_dates = []
album_release_date_precisions = []
popularity = []
is_explicit = []
durations_ms = []

artist_ids.append(results['tracks']['items'][0]['artists'][0]['id'])
artist_names.append(results['tracks']['items'][0]['artists'][0]['name'])
track_ids.append(results['tracks']['items'][0]['id'])
track_href_1.append(results['tracks']['items'][0]['href'])
track_names.append(results['tracks']['items'][0]['name'])
popularity.append(results['tracks']['items'][0]['popularity'])
is_explicit.append(results['tracks']['items'][0]['explicit'])
durations_ms.append(results['tracks']['items'][0]['duration_ms'])
album_release_dates.append(results['tracks']['items'][0]['album']['release_date'])
album_release_date_precisions.append(results['tracks']['items'][0]['album']['release_date_precision'])


tracks_df_fav = pd.DataFrame(
{'artist_id':artist_ids
,'artist_name':artist_names
, 'track_id':track_ids
, 'track_name':track_names
, 'album_release_date':album_release_dates
, 'album_release_date_precision':album_release_date_precisions
, 'is_explicit': is_explicit
, 'durations_ms':durations_ms
, 'popularity':popularity
, 'track_href_1':track_href_1})

merged_fav = pd.merge(tracks_df_fav, audio_features_df, left_on='track_href_1', right_on='track_href')
merged_fav.columns
merged_fav = merged_fav.drop(columns=['track_href_1'])

new_df = pd.concat([df_tf, merged_fav], axis=0, ignore_index=True)

audio_features_model_on = ['is_explicit','popularity', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'time_signature']
features_df = new_df[audio_features_model_on]

km100 = KMeans(100)
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features_df)
km100.fit(scaled_features)

tracks_clustered_df = new_df.copy()
tracks_clustered_df['cluster_km100'] = km100.labels_

# Assuming `merged_fav` contains the new track data
new_track_id = merged_fav['track_id'].iloc[-1]  # Get the track_id of the newly added track

# Get the cluster number for the new track
cluster_number = tracks_clustered_df[tracks_clustered_df['track_id'] == new_track_id]['cluster_km100'].iloc[0]


#put the path to the cluster of the new song
recommendation = tracks_clustered_df[tracks_clustered_df['cluster_km100'] == cluster_number].sample(1)

from IPython.display import IFrame

IFrame(src="https://open.spotify.com/embed/track/"+ recommendation['track_id'].iloc[0] +"?utm_source=generator",

       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

In [5]:
merged_fav['key'].iloc[0]

7

In [6]:
### with Pickling


audio_features_model_on = ['is_explicit','popularity', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'time_signature']
features_df_fav = merged_fav[audio_features_model_on]


# Load the scaler and the KMeans model
with open('scaler.pickle', 'rb') as handle:
    scaler = pickle.load(handle)
with open('model_km100.pickle', 'rb') as handle:
    km_100 = pickle.load(handle)

# Assuming `new_song_features` is your new data
new_song_scaled_features = scaler.transform(features_df_fav)

# Use the loaded KMeans model to make predictions
prediction = km_100.predict(new_song_scaled_features[[-1]])[0]


recommendation2 = df[df['cluster_km100'] == prediction].sample(1)
recommendation2['track_id'].iloc[0]

from IPython.display import IFrame

IFrame(src="https://open.spotify.com/embed/track/"+ recommendation2['track_id'].iloc[0] +"?utm_source=generator",
       #src="spotify:track:3rTIcUMMP2Ez33DfjJlb9e:autoplay:true",
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

/Users/Barbara/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.4.1.post1 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/Barbara/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator KMeans from version 1.4.1.post1 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [9]:

with open('/Users/Barbara/Desktop/Ironhack/Labs/Week_6/spotipy-api/final.pkl', 'rb') as handle:
    data_loaded = pickle.load(handle)

# Extract objects from the loaded data
scaler_L1 = data_loaded['scaler_L1']
scaler_L2 = data_loaded['scaler_L2']
scaler_L3 = data_loaded['scaler_L3']
scaler_L4 = data_loaded['scaler_L4']
kmeans_L1 = data_loaded['kmeans_L1']
kmeans_L2_models = data_loaded['kmeans_L2_models']
kmeans_L3_models = data_loaded['kmeans_L3_models']
kmeans_L4_models = data_loaded['kmeans_L4_models']
features_df = data_loaded['features_df']

# Select features for each level
features_L1 = ['valence', 'energy', 'acousticness']
features_L2 = ['danceability', 'tempo', 'liveness']
features_L3 = ['instrumentalness', 'speechiness', 'loudness']
features_L4 = ['key', 'mode']

# Scale the input features
input_features_scaled_L1 = scaler_L1.transform(audio_features_df[features_L1])
input_features_scaled_L2 = scaler_L2.transform(audio_features_df[features_L2])
input_features_scaled_L3 = scaler_L3.transform(audio_features_df[features_L3])
input_features_scaled_L4 = scaler_L4.transform(audio_features_df[features_L4])

# Predict cluster at Level 1
cluster_L1 = kmeans_L1.predict(input_features_scaled_L1)[0]

# Predict cluster at Level 2
kmeans_L2 = kmeans_L2_models[cluster_L1]
cluster_L2 = kmeans_L2.predict(input_features_scaled_L2)[0]

# Predict cluster at Level 3
kmeans_L3 = kmeans_L3_models[cluster_L2]
cluster_L3 = kmeans_L3.predict(input_features_scaled_L3)[0]

# Predict cluster at Level 4
kmeans_L4 = kmeans_L4_models[cluster_L3]
cluster_L4 = kmeans_L4.predict(input_features_scaled_L4)[0]

print(f"Cluster at Level 1: {cluster_L1}")
print(f"Cluster at Level 2: {cluster_L2}")
print(f"Cluster at Level 3: {cluster_L3}")
print(f"Cluster at Level 4: {cluster_L4}")


features_df=df_tf

# Grouping the audio features into cluster levels
features_L1 = ['valence', 'energy', 'acousticness']
features_L2 = ['danceability', 'tempo', 'liveness']
features_L3 = ['instrumentalness', 'speechiness', 'loudness']
features_L4 = ['key', 'mode']

# Create separate scalers for each level
scaler_L1 = StandardScaler()
scaler_L2 = StandardScaler()
scaler_L3 = StandardScaler()
scaler_L4 = StandardScaler()

def perform_clustering(data, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(data)
    return clusters, kmeans

# Level 1 clustering
level_L1_scaled = scaler_L1.fit_transform(features_df[features_L1])
clusters_L1, kmeans_L1 = perform_clustering(level_L1_scaled, n_clusters=10)

features_df['cluster_L1'] = clusters_L1

# Initialize the final clusters list
final_clusters = np.zeros(features_df.shape[0])

# Secondary clustering (Level 2)
cluster_counter = 0
kmeans_L2_models = {}
for cluster_L1_label in np.unique(clusters_L1):
    level_2_data = features_df[features_df['cluster_L1'] == cluster_L1_label][features_L2]
    level_2_scaled = scaler_L2.fit_transform(level_2_data)
    clusters_L2, kmeans_L2 = perform_clustering(level_2_scaled, n_clusters=5)
    
    features_df.loc[features_df['cluster_L1'] == cluster_L1_label, 'cluster_L2'] = clusters_L2 + cluster_counter * 5
    kmeans_L2_models[cluster_L1_label] = kmeans_L2
    cluster_counter += 1

# Clustering for Level 3
cluster_counter = 0
kmeans_L3_models = {}
for cluster_L2_label in np.unique(features_df['cluster_L2']):
    level_3_data = features_df[features_df['cluster_L2'] == cluster_L2_label][features_L3]
    level_3_scaled = scaler_L3.fit_transform(level_3_data)
    clusters_L3, kmeans_L3 = perform_clustering(level_3_scaled, n_clusters=2)
    
    features_df.loc[features_df['cluster_L2'] == cluster_L2_label, 'cluster_L3'] = clusters_L3 + cluster_counter * 2
    kmeans_L3_models[cluster_L2_label] = kmeans_L3
    cluster_counter += 1

# Clustering for Level 4
cluster_counter = 0
kmeans_L4_models = {}
for cluster_L3_label in np.unique(features_df['cluster_L3']):
    level_4_data = features_df[features_df['cluster_L3'] == cluster_L3_label][features_L4]
    level_4_scaled = scaler_L4.fit_transform(level_4_data)
    clusters_L4, kmeans_L4 = perform_clustering(level_4_scaled, n_clusters=2)
    
    features_df.loc[features_df['cluster_L3'] == cluster_L3_label, 'cluster_L4'] = clusters_L4 + cluster_counter
    kmeans_L4_models[cluster_L3_label] = kmeans_L4
    cluster_counter += 1

input_clusters = {
    'L1': cluster_L1,
    'L2': cluster_L2,
    'L3': cluster_L3,
    'L4': cluster_L4
}
    # Assuming input_clusters are already computed as cluster_L1, cluster_L2, cluster_L3, cluster_L4

# Function to calculate weighted common clusters
def calculate_weighted_common_clusters(row):
    weighted_score = 0
    if row['cluster_L4'] == input_clusters['L4']:
        weighted_score += 4  # Weight for level 4
    if row['cluster_L3'] == input_clusters['L3']:
        weighted_score += 3  # Weight for level 3
    if row['cluster_L2'] == input_clusters['L2']:
        weighted_score += 2  # Weight for level 2
    if row['cluster_L1'] == input_clusters['L1']:
        weighted_score += 1  # Weight for level 1
    return weighted_score

# Calculate weighted common clusters for each song
features_df['weighted_common_clusters'] = features_df.apply(calculate_weighted_common_clusters, axis=1)

# Find song with the maximum weighted common clusters
top_recommendation = features_df.loc[features_df['weighted_common_clusters'].idxmax()]

# Optionally, filter out input song itself from recommendations


# Now `top_recommendation` is the song from `features_df` that has the most clusters in common with the input song,
# with bias towards higher levels of clustering
top_recommendation


IFrame(src="https://open.spotify.com/embed/track/"+ top_recommendation['track_id'] +"?utm_source=generator",
       #src="spotify:track:3rTIcUMMP2Ez33DfjJlb9e:autoplay:true",
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )



Cluster at Level 1: 0
Cluster at Level 2: 0
Cluster at Level 3: 1
Cluster at Level 4: 0


In [8]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pickle

# Load data
with open('/Users/Barbara/Desktop/Ironhack/Labs/Week_6/spotipy-api/final.pkl', 'rb') as handle:
    data_loaded = pickle.load(handle)

# Extract objects from the loaded data
scaler_L1 = data_loaded['scaler_L1']
scaler_L2 = data_loaded['scaler_L2']
scaler_L3 = data_loaded['scaler_L3']
scaler_L4 = data_loaded['scaler_L4']
kmeans_L1 = data_loaded['kmeans_L1']
kmeans_L2_models = data_loaded['kmeans_L2_models']
kmeans_L3_models = data_loaded['kmeans_L3_models']
kmeans_L4_models = data_loaded['kmeans_L4_models']
features_df = data_loaded['features_df']

# Define features for each level
features_L1 = ['valence', 'energy', 'acousticness']
features_L2 = ['danceability', 'tempo', 'liveness']
features_L3 = ['instrumentalness', 'speechiness', 'loudness']
features_L4 = ['key', 'mode']

# Load and scale the input features
input_features_scaled_L1 = scaler_L1.transform(features_df[features_L1])
input_features_scaled_L2 = scaler_L2.transform(features_df[features_L2])
input_features_scaled_L3 = scaler_L3.transform(features_df[features_L3])
input_features_scaled_L4 = scaler_L4.transform(features_df[features_L4])

# Predict clusters at each level
cluster_L1 = kmeans_L1.predict(input_features_scaled_L1)[0]
kmeans_L2 = kmeans_L2_models[cluster_L1]
cluster_L2 = kmeans_L2.predict(input_features_scaled_L2)[0]
kmeans_L3 = kmeans_L3_models[cluster_L2]
cluster_L3 = kmeans_L3.predict(input_features_scaled_L3)[0]
kmeans_L4 = kmeans_L4_models[cluster_L3]
cluster_L4 = kmeans_L4.predict(input_features_scaled_L4)[0]

print(f"Cluster at Level 1: {cluster_L1}")
print(f"Cluster at Level 2: {cluster_L2}")
print(f"Cluster at Level 3: {cluster_L3}")
print(f"Cluster at Level 4: {cluster_L4}")

# Verify clusters
print("Unique Clusters at Level 1:", np.unique(features_df['cluster_L1']))
print("Unique Clusters at Level 2:", np.unique(features_df['cluster_L2']))
print("Unique Clusters at Level 3:", np.unique(features_df['cluster_L3']))
print("Unique Clusters at Level 4:", np.unique(features_df['cluster_L4']))


Cluster at Level 1: 1
Cluster at Level 2: 1
Cluster at Level 3: 0
Cluster at Level 4: 0
Unique Clusters at Level 1: [0 1 2 3 4 5 6 7 8 9]
Unique Clusters at Level 2: [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49.]
Unique Clusters at Level 3: [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53.
 54. 55. 56. 57. 58. 59. 60. 61. 62. 63. 64. 65. 66. 67. 68. 69. 70. 71.
 72. 73. 74. 75. 76. 77. 78. 79. 80. 81. 82. 83. 84. 85. 86. 87. 88. 89.
 90. 91. 92. 93. 94. 95. 96. 97. 98. 99.]
Unique Clusters at Level 4: [  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.